In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [12]:
# Log-ratio transformations
def clr_inv(p):
    z = tf.math.log(p)
    return z - tf.reduce_mean(z, axis=1)[:, tf.newaxis]

def clr_forward(z, axis=1):
    return tf.nn.softmax(z, axis=axis)

In [13]:
def wide_resnet(num_classes, depth=28, width=2):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(16, (3, 3), padding="same")(inputs)
    for _ in range(depth // 6):
        x = layers.Conv2D(16 * width, (3, 3), padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)  # Use Keras's ReLU layer
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes)(x)
    return models.Model(inputs, outputs)

In [14]:
# Dataset loader combining TF CIFAR datasets with custom labels
def load_combined_dataset(cifar_version, label_file, num_classes):
    """
    Combine CIFAR image data from TensorFlow with labels from a .npy file.

    Args:
        cifar_version (str): "cifar10" or "cifar100" to select the dataset.
        label_file (str): Path to the .npy file containing labels.
        num_classes (int): Number of classes (10 or 100).

    Returns:
        tf.data.Dataset: Dataset ready for training/testing.
    """
    # Load image data
    if cifar_version == "cifar10":
        (x_train, _), _ = tf.keras.datasets.cifar10.load_data()
    elif cifar_version == "cifar100":
        (x_train, _), _ = tf.keras.datasets.cifar100.load_data()
    else:
        raise ValueError("cifar_version must be 'cifar10' or 'cifar100'.")

    # Load labels from .npy file
    label_data = np.load(label_file, allow_pickle=True).item()
    if 'clean_label' not in label_data:
        raise ValueError("Label file must contain 'clean_label' key.")

    # Normalize image data and convert labels
    x_train = x_train.astype(np.float32) / 255.0
    y_train = tf.keras.utils.to_categorical(label_data['clean_label'], num_classes)

    # Create TensorFlow dataset
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.shuffle(10000).batch(64)
    return train_ds

In [15]:
# Gaussian noise (manual implementation)
def add_gaussian_noise(inputs, mean=0.0, stddev=0.1):
    noise = tf.random.normal(shape=tf.shape(inputs), mean=mean, stddev=stddev)
    return inputs + noise

In [16]:
# Training step
@tf.function
def train_step(model, images, labels, optimizer):
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        smoothed_targets = clr_forward(labels)
        logit_targets = clr_inv(smoothed_targets)
        loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(logit_targets, logits))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [21]:
import pandas as pd

def train_model(train_ds, epochs=10, num_classes=10):
    model = wide_resnet(num_classes)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # Define metrics
    accuracy_metric = tf.keras.metrics.CategoricalAccuracy()
    precision_metric = tf.keras.metrics.Precision()
    recall_metric = tf.keras.metrics.Recall()

    # DataFrame to store metrics
    metrics_df = pd.DataFrame(columns=["Epoch", "Loss", "Accuracy", "Precision", "Recall", "F1-Score"])

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")
        epoch_loss_sum = 0
        num_steps = 0

        # Reset metrics at the start of each epoch
        accuracy_metric.reset_states()
        precision_metric.reset_states()
        recall_metric.reset_states()

        for step, (images, labels) in enumerate(train_ds):
            # Perform training step
            loss = train_step(model, images, labels, optimizer)
            
            # Update metrics
            epoch_loss_sum += loss.numpy()
            num_steps += 1
            predictions = model(images, training=False)
            accuracy_metric.update_state(labels, predictions)
            precision_metric.update_state(labels, predictions)
            recall_metric.update_state(labels, predictions)

            if step % 10 == 0:
                print(f"Step {step}, Loss: {loss.numpy()}")

        # Compute metrics for the epoch
        avg_loss = epoch_loss_sum / num_steps
        epoch_accuracy = accuracy_metric.result().numpy()
        epoch_precision = precision_metric.result().numpy()
        epoch_recall = recall_metric.result().numpy()
        epoch_f1_score = 2 * (epoch_precision * epoch_recall) / (epoch_precision + epoch_recall + 1e-7)

        # Display metrics
        print(f"Epoch {epoch + 1} - Average Loss: {avg_loss:.4f}, "
              f"Accuracy: {epoch_accuracy:.4f}, "
              f"Precision: {epoch_precision:.4f}, "
              f"Recall: {epoch_recall:.4f}, "
              f"F1-Score: {epoch_f1_score:.4f}")

        # Save metrics to DataFrame
        metrics_df = metrics_df.append({
            "Epoch": epoch + 1,
            "Loss": avg_loss,
            "Accuracy": epoch_accuracy,
            "Precision": epoch_precision,
            "Recall": epoch_recall,
            "F1-Score": epoch_f1_score
        }, ignore_index=True)

        # Save checkpoint with proper extension
        os.makedirs("checkpoints", exist_ok=True)
        model.save_weights(f"checkpoints/sgn_epoch_{epoch}.weights.h5")
        print(f"Checkpoint saved for epoch {epoch + 1}")

    # Save metrics to a CSV file
    metrics_df.to_csv("training_metrics.csv", index=False)
    print("Metrics saved to 'training_metrics.csv'")

In [20]:
if __name__ == "__main__":
    # Paths to label files
    cifar10_label_file = "./data/CIFAR-10_human_ordered.npy"
    cifar100_label_file = "./data/CIFAR-100_human_ordered.npy"

    # Load CIFAR-10 dataset for training
    train_ds_cifar10 = load_combined_dataset("cifar10", cifar10_label_file, num_classes=10)

    # Train on CIFAR-10 (CIFAR-100 can be loaded similarly)
    train_model(train_ds_cifar10, epochs=10, num_classes=10)

Epoch 1/10
Step 0, Loss: -0.30526530742645264
Step 10, Loss: -1.2208292484283447
Step 20, Loss: -0.24567633867263794
Step 30, Loss: 0.16392982006072998
Step 40, Loss: 1.118992567062378
Step 50, Loss: -0.6978888511657715
Step 60, Loss: -1.0975253582000732
Step 70, Loss: -0.07976388931274414
Step 80, Loss: 1.1383073329925537
Step 90, Loss: -0.7111796140670776
Step 100, Loss: -1.5299311876296997
Step 110, Loss: 0.1551665961742401
Step 120, Loss: -0.9019607305526733
Step 130, Loss: 0.547715425491333
Step 140, Loss: 0.0065004825592041016
Step 150, Loss: 1.1441545486450195
Step 160, Loss: 0.8311232328414917
Step 170, Loss: -0.36357927322387695
Step 180, Loss: -1.7897570133209229
Step 190, Loss: -1.4610923528671265
Step 200, Loss: 0.7477368116378784
Step 210, Loss: -0.05420875549316406
Step 220, Loss: -0.8020827174186707
Step 230, Loss: 0.5844733119010925
Step 240, Loss: -0.35730183124542236
Step 250, Loss: -0.8903634548187256
Step 260, Loss: 0.557440996170044
Step 270, Loss: 0.40324735641479